In [1]:
import yfinance as yf  
import pandas as pd

In [2]:
import numpy as np

In [ ]:
# from src.utils import nan_info

In [ ]:
from src.data_extraction import Indian_stocks_df
from src.data_extraction import nan_info

In [4]:
start_date = "2008-01-02"
end_date = '2022-12-31'

In [5]:
df = Indian_stocks_df(start_date, end_date)

$IREDA.NS: possibly delisted; no price data found  (1d 2008-01-02 -> 2022-12-31) (Yahoo error = "Data doesn't exist for startDate = 1199212200, endDate = 1672425000")
$MANKIND.NS: possibly delisted; no price data found  (1d 2008-01-02 -> 2022-12-31) (Yahoo error = "Data doesn't exist for startDate = 1199212200, endDate = 1672425000")
$NUVAMA.NS: possibly delisted; no price data found  (1d 2008-01-02 -> 2022-12-31) (Yahoo error = "Data doesn't exist for startDate = 1199212200, endDate = 1672425000")
$ADANIENSOL.NS: possibly delisted; no price data found  (1d 2008-01-02 -> 2022-12-31) (Yahoo error = "Data doesn't exist for startDate = 1199212200, endDate = 1672425000")
$TATATECH.NS: possibly delisted; no price data found  (1d 2008-01-02 -> 2022-12-31) (Yahoo error = "Data doesn't exist for startDate = 1199212200, endDate = 1672425000")
$JIOFIN.NS: possibly delisted; no price data found  (1d 2008-01-02 -> 2022-12-31) (Yahoo error = "Data doesn't exist for startDate = 1199212200, endDate =

In [6]:
df.to_csv("datas/Raw_data.csv")

In [7]:
print('The length of the raw data set: ',df.shape)
df.head()

The length of the raw data set:  (3698, 209)


,360ONE.NS,ABB.NS,ABCAPITAL.NS,ADANIENSOL.NS,ADANIENT.NS,ADANIGREEN.NS,ADANIPORTS.NS,ALKEM.NS,AMBER.NS,AMBUJACEM.NS,...,UNIONBANK.NS,UNITDSPR.NS,UNOMINDA.NS,UPL.NS,VBL.NS,VEDL.NS,VOLTAS.NS,WIPRO.NS,YESBANK.NS,ZYDUSLIFE.NS
date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,NaN,1372.092285,NaN,NaN,97.115074,NaN,261.489990,NaN,NaN,147.949997,...,229.149994,NaN,8.261768,124.849998,NaN,192.797501,252.100006,57.526878,53.150002,38.412254
2008-01-03,NaN,1362.060425,NaN,NaN,97.532753,NaN,255.160004,NaN,NaN,145.000000,...,227.350006,NaN,7.926787,125.449997,NaN,190.345001,254.149994,55.822502,52.139999,38.436966
2008-01-04,NaN,1370.639648,NaN,NaN,97.237694,NaN,248.070007,NaN,NaN,143.350006,...,229.050003,NaN,7.502256,127.833336,NaN,189.872498,254.899994,55.873127,52.009998,38.195999
2008-01-07,NaN,1361.878784,NaN,NaN,95.977013,NaN,245.449997,NaN,NaN,142.550003,...,225.899994,NaN,7.296624,128.783340,NaN,187.377502,248.899994,54.275627,52.680000,38.072430
2008-01-08,NaN,1360.562378,NaN,NaN,93.539932,NaN,240.960007,NaN,NaN,142.000000,...,224.550003,NaN,7.032950,125.633331,NaN,185.882507,241.050003,55.057503,52.040001,36.978821


In [ ]:
# print('-------- Nan per column ------')
# print(df.isna().sum(),'\n')

# print('-------------------------------')

# print('Columns with all Nan : ',df.isna().all().sum())

# print('Columns with atleast 1 Zan : ', df.isna().any().sum())
nan_info(df)

-------- Nan per column ------
360ONE.NS        2882
ABB.NS              1
ABCAPITAL.NS     2379
ADANIENSOL.NS    3698
ADANIENT.NS         1
                 ... 
VEDL.NS             1
VOLTAS.NS           1
WIPRO.NS            1
YESBANK.NS          1
ZYDUSLIFE.NS        1
Length: 209, dtype: int64 

-------------------------------
Columns with all Nan :  8
Columns with atleast 1 Zan :  208


In [9]:
nan_summary = (
    df.isna()
      .sum()
      .loc[lambda x: x > 100]
      .to_frame(name="nan_count")
)
nan_summary

,nan_count
360ONE.NS,2882
ABCAPITAL.NS,2379
ADANIENSOL.NS,3698
ADANIGREEN.NS,2576
ALKEM.NS,1962
...,...
TATAMOTORS.NS,3698
TATATECH.NS,3698
TIINDIA.NS,2421
UNITDSPR.NS,3698


We see that there are many stocks that have empty data, we cannot remove them irresposibly because we will loose importamt information. We could start by keeping stocks that have atleast 90% of data available and then we could fill the empty data points with the previous values. But, we are carful here also to only fill atmost 3 consecutive data points with the prvious values, this is because if we are dealing withstocks that did not have an IPO before a time period or it got delisted after some time then filling without limit will cause us to have long periods of time with same price leading to wrong results.

In [10]:
coverage = df.notna().mean()

keep_cols = coverage[coverage > 0.9].index
df = df[keep_cols]

df = df.ffill(limit=3)

print(df.shape)


(3698, 127)


In [ ]:
# print('-------- Nan per column------')
# print(df.isna().sum(),'\n')

# print('-------------------------------')

# print('Columns with all Nan : ',df.isna().all().sum())

# print('Columns with atleast 1 Nan : ', df.isna().any().sum())
nan_info(df)

-------- Nan per column------
ABB.NS           0
ADANIENT.NS      0
ADANIPORTS.NS    0
AMBUJACEM.NS     0
APOLLOHOSP.NS    0
                ..
VEDL.NS          0
VOLTAS.NS        0
WIPRO.NS         0
YESBANK.NS       0
ZYDUSLIFE.NS     0
Length: 127, dtype: int64 

-------------------------------
Columns with all Nan :  0
Columns with atleast 1 Nan :  3


Okay apart from the data we removed we only have 13 columns with empty data points <= 10 % we could remove these stocks and do our analysis because we are still left with more than a 100 stocks which will still provide us with good insights without the unnessecary complexity

In [12]:
nan_counts = df.isna().sum()
nan_cols = nan_counts[nan_counts >= 1]

print(nan_cols)

NMDC.NS        43
RECLTD.NS      49
TITAGARH.NS    73
dtype: int64


In [13]:
df = df.dropna(axis=1)

In [ ]:
# print('-------- Nan per column------')
# print(df.isna().sum(),'\n')

# print('-------------------------------')

# print('Columns with all Nan : ',df.isna().all().sum())

# print('Columns with atleast 1 Nan : ', df.isna().any().sum()
nan_info(df)

-------- Nan per column------
ABB.NS           0
ADANIENT.NS      0
ADANIPORTS.NS    0
AMBUJACEM.NS     0
APOLLOHOSP.NS    0
                ..
VEDL.NS          0
VOLTAS.NS        0
WIPRO.NS         0
YESBANK.NS       0
ZYDUSLIFE.NS     0
Length: 124, dtype: int64 

-------------------------------
Columns with all Nan :  0
Columns with atleast 1 Nan :  0


In [15]:
print('The length of the cleaned data set: ',df.shape)
df.head()

The length of the cleaned data set:  (3698, 124)


,ABB.NS,ADANIENT.NS,ADANIPORTS.NS,AMBUJACEM.NS,APOLLOHOSP.NS,ASHOKLEY.NS,ASIANPAINT.NS,ASTRAL.NS,AUROPHARMA.NS,AXISBANK.NS,...,TVSMOTOR.NS,ULTRACEMCO.NS,UNIONBANK.NS,UNOMINDA.NS,UPL.NS,VEDL.NS,VOLTAS.NS,WIPRO.NS,YESBANK.NS,ZYDUSLIFE.NS
date,,,,,,,,,,,,,,,,,,,,,
2008-01-02,1372.092285,97.115074,261.489990,147.949997,294.700012,12.9750,113.629997,10.334252,54.599998,206.929993,...,36.025002,992.750000,229.149994,8.261768,124.849998,192.797501,252.100006,57.526878,53.150002,38.412254
2008-01-03,1362.060425,97.532753,255.160004,145.000000,285.825012,13.1500,113.904999,10.305002,53.709999,207.169998,...,35.700001,987.000000,227.350006,7.926787,125.449997,190.345001,254.149994,55.822502,52.139999,38.436966
2008-01-04,1370.639648,97.237694,248.070007,143.350006,285.500000,13.5000,114.089996,9.805502,53.244999,210.979996,...,35.474998,977.299988,229.050003,7.502256,127.833336,189.872498,254.899994,55.873127,52.009998,38.195999
2008-01-07,1361.878784,95.977013,245.449997,142.550003,279.750000,14.1000,121.239998,9.922502,53.020000,218.029999,...,38.724998,988.450012,225.899994,7.296624,128.783340,187.377502,248.899994,54.275627,52.680000,38.072430
2008-01-08,1360.562378,93.539932,240.960007,142.000000,273.799988,13.5125,124.820000,9.351002,51.980000,213.149994,...,36.125000,971.500000,224.550003,7.032950,125.633331,185.882507,241.050003,55.057503,52.040001,36.978821


In [16]:
df.to_csv("datas/Clean_data.csv")

We, want to work with the log returns because of it is approximately normally distributed and using the symmetric and additive property of log returns

In [17]:
df = np.log(df/df.shift())
df.dropna()

,ABB.NS,ADANIENT.NS,ADANIPORTS.NS,AMBUJACEM.NS,APOLLOHOSP.NS,ASHOKLEY.NS,ASIANPAINT.NS,ASTRAL.NS,AUROPHARMA.NS,AXISBANK.NS,...,TVSMOTOR.NS,ULTRACEMCO.NS,UNIONBANK.NS,UNOMINDA.NS,UPL.NS,VEDL.NS,VOLTAS.NS,WIPRO.NS,YESBANK.NS,ZYDUSLIFE.NS
date,,,,,,,,,,,,,,,,,,,,,
2008-01-03,-0.007338,0.004292,-0.024505,-0.020141,-0.030578,0.013397,0.002417,-0.002834,-0.016435,0.001159,...,-0.009062,-0.005809,-0.007886,-0.041391,0.004794,-0.012802,0.008099,-0.030075,-0.019186,0.000643
2008-01-04,0.006279,-0.003030,-0.028180,-0.011445,-0.001138,0.026268,0.001623,-0.049686,-0.008695,0.018224,...,-0.006323,-0.009876,0.007450,-0.055044,0.018820,-0.002485,0.002947,0.000906,-0.002496,-0.006289
2008-01-07,-0.006412,-0.013050,-0.010618,-0.005596,-0.020346,0.043485,0.060784,0.011861,-0.004235,0.032869,...,0.087657,0.011344,-0.013848,-0.027792,0.007404,-0.013227,-0.023820,-0.029008,0.012800,-0.003240
2008-01-08,-0.000967,-0.025720,-0.018462,-0.003866,-0.021498,-0.042560,0.029101,-0.059322,-0.019810,-0.022637,...,-0.069500,-0.017297,-0.005994,-0.036806,-0.024764,-0.008011,-0.032047,0.014303,-0.012223,-0.029145
2008-01-09,-0.004414,-0.041145,-0.020587,-0.000705,-0.017872,0.004615,-0.036967,-0.001445,-0.025032,0.027623,...,-0.013236,-0.012168,-0.024570,0.013816,0.034551,-0.038651,-0.010425,0.002857,-0.000384,0.002337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-26,0.016243,0.020262,0.014936,0.045562,-0.003431,0.024719,-0.000605,0.034172,-0.000686,0.024084,...,0.004985,0.023543,0.172279,0.004776,0.009864,0.037507,0.011188,0.007518,0.116190,0.001070
2022-12-27,0.006585,0.014133,0.013677,0.017640,-0.008630,0.003885,0.018335,0.006534,0.010240,0.003762,...,0.027677,0.004682,0.030677,-0.000667,-0.002649,0.038265,0.014203,0.015904,0.027675,-0.003452
2022-12-28,-0.006695,0.007413,-0.008356,-0.009408,-0.011947,0.013653,0.003560,0.000456,-0.003402,-0.008078,...,0.036350,-0.007542,-0.041326,0.003427,0.010001,-0.008979,0.000000,0.003099,0.000000,-0.009706


In [18]:
df.to_csv('datas/Log_returns_data.csv')